# Brief Tour of the Standard Library — Part II

Source: [Python Documentation](https://docs.python.org/3/tutorial/stdlib2.html)

This second tour covers more advanced modules that support professional programming needs. These modules rarely occur in small scripts.

## 11.1. Output Formatting

The `reprlib` module provides a version of `repr()` customized for abbreviated displays of large or deeply nested containers:

In [ ]:
import reprlib
reprlib.repr(set('supercalifragilisticexpialidocious'))

The `pprint` module offers more sophisticated control over printing both built-in and user defined objects in a way that is readable by the interpreter:

In [ ]:
import pprint
t = [[[['black', 'cyan'], 'white', ['green', 'red']], [['magenta',
    'yellow'], 'blue']]]

pprint.pprint(t, width=30)

The `textwrap` module formats paragraphs of text to fit a given screen width:

In [ ]:
import textwrap
doc = """The wrap() method is just like fill() except that it returns
a list of strings instead of one big string with newlines to separate
the wrapped lines."""

print(textwrap.fill(doc, width=40))

The `locale` module accesses a database of culture specific data formats. The grouping attribute of locale's format function provides a direct way of formatting numbers with group separators:

In [ ]:
import locale
# Note: This example may work differently on different systems
# locale.setlocale(locale.LC_ALL, 'English_United States.1252')
# conv = locale.localeconv()  # get a mapping of conventions
# x = 1234567.8
# locale.format_string("%d", x, grouping=True)

## 11.2. Templating

The `string` module includes a versatile `Template` class with a simplified syntax suitable for editing by end-users. The format uses placeholder names formed by `$` with valid Python identifiers:

In [ ]:
from string import Template
t = Template('${village}folk send $$10 to $cause.')
t.substitute(village='Nottingham', cause='the ditch fund')

The `substitute()` method raises a `KeyError` when a placeholder is not supplied. The `safe_substitute()` method will leave placeholders unchanged if data is missing:

In [ ]:
t = Template('Return the $item to $owner.')
d = dict(item='unladen swallow')
# t.substitute(d)  # This would raise KeyError
t.safe_substitute(d)

Template subclasses can specify a custom delimiter. For example, a batch renaming utility for a photo browser may use percent signs:

In [ ]:
import time, os.path
photofiles = ['img_1074.jpg', 'img_1076.jpg', 'img_1077.jpg']

class BatchRename(Template):
    delimiter = '%'

# Example format: 'Ashley_%n%f'
fmt = 'Ashley_%n%f'
t = BatchRename(fmt)
date = time.strftime('%d%b%y')

for i, filename in enumerate(photofiles):
    base, ext = os.path.splitext(filename)
    newname = t.substitute(d=date, n=i, f=ext)
    print('{0} --> {1}'.format(filename, newname))

## 11.3. Working with Binary Data Record Layouts

The `struct` module provides `pack()` and `unpack()` functions for working with variable length binary record formats. Pack codes `"H"` and `"I"` represent two and four byte unsigned numbers respectively. The `"<"` indicates that they are standard size and in little-endian byte order:

In [ ]:
import struct

# Example: Pack and unpack binary data
data = struct.pack('<IIIHH', 12345, 67890, 11111, 22, 33)
print("Packed data:", data)

# Unpack the data
fields = struct.unpack('<IIIHH', data)
print("Unpacked:", fields)

## 11.4. Multi-threading

Threading is a technique for decoupling tasks which are not sequentially dependent. The following code shows how the high level `threading` module can run tasks in background while the main program continues to run:

In [ ]:
import threading, zipfile, time

class AsyncZip(threading.Thread):
    def __init__(self, infile, outfile):
        threading.Thread.__init__(self)
        self.infile = infile
        self.outfile = outfile

    def run(self):
        # Simulate some work
        time.sleep(1)
        print(f'Finished background processing of: {self.infile}')

# Example usage (commented out for notebook)
# background = AsyncZip('mydata.txt', 'myarchive.zip')
# background.start()
# print('The main program continues to run in foreground.')
# background.join()  # Wait for the background task to finish
# print('Main program waited until background was done.')

**Note:** Applications using `Queue` objects for inter-thread communication and coordination are easier to design, more readable, and more reliable.

## 11.5. Logging

The `logging` module offers a full featured and flexible logging system. At its simplest, log messages are sent to a file or to `sys.stderr`:

In [3]:
import logging

logging.basicConfig(level=logging.DEBUG)
logging.debug('Debugging information')
logging.info('Informational message')
logging.warning('Warning:config file %s not found', 'server.conf')
logging.error('Error occurred')
logging.critical('Critical error -- shutting down')

ERROR:root:Error occurred
CRITICAL:root:Critical error -- shutting down


By default, informational and debugging messages are suppressed and the output is sent to standard error. The logging system can be configured directly from Python or can be loaded from a user editable configuration file.

## 11.6. Weak References

Python does automatic memory management (reference counting for most objects and garbage collection to eliminate cycles). The `weakref` module provides tools for tracking objects without creating a reference:

In [ ]:
import weakref, gc

class A:
    def __init__(self, value):
        self.value = value
    def __repr__(self):
        return str(self.value)

a = A(10)  # create a reference
d = weakref.WeakValueDictionary()
d['primary'] = a  # does not create a reference
print(d['primary'])  # fetch the object if it is still alive

In [ ]:
del a  # remove the one reference
gc.collect()  # run garbage collection right away

try:
    print(d['primary'])  # entry was automatically removed
except KeyError:
    print("KeyError: 'primary' - entry was automatically removed")

## 11.7. Tools for Working with Lists

The `array` module provides an `array` object that is like a list that stores only homogeneous data and stores it more compactly:

In [ ]:
from array import array
a = array('H', [4000, 10, 700, 22222])
print("Sum:", sum(a))
print("Slice:", a[1:3])

The `collections` module provides a `deque` object that is like a list with faster appends and pops from the left side:

In [ ]:
from collections import deque
d = deque(["task1", "task2", "task3"])
d.append("task4")
print("Handling", d.popleft())

Example of using deque for breadth-first search:

In [ ]:
# Example breadth-first search structure
def breadth_first_search(starting_node, gen_moves, is_goal):
    """Template for breadth-first search"""
    unsearched = deque([starting_node])
    while unsearched:
        node = unsearched.popleft()
        for m in gen_moves(node):
            if is_goal(m):
                return m
            unsearched.append(m)
    return None

The `bisect` module provides functions for manipulating sorted lists:

In [ ]:
import bisect
scores = [(100, 'perl'), (200, 'tcl'), (400, 'lua'), (500, 'python')]
bisect.insort(scores, (300, 'ruby'))
scores

The `heapq` module provides functions for implementing heaps based on regular lists:

In [ ]:
from heapq import heapify, heappop, heappush
data = [1, 3, 5, 7, 9, 2, 4, 6, 8, 0]
heapify(data)  # rearrange the list into heap order
heappush(data, -5)  # add a new entry
[heappop(data) for i in range(3)]  # fetch the three smallest entries

## 11.8. Decimal Floating-Point Arithmetic

The `decimal` module offers a `Decimal` datatype for decimal floating-point arithmetic. This is especially helpful for:

- Financial applications and other uses which require exact decimal representation
- Control over precision
- Control over rounding to meet legal or regulatory requirements
- Tracking of significant decimal places
- Applications where the user expects the results to match calculations done by hand

In [ ]:
from decimal import *

# Calculating a 5% tax on a 70 cent phone charge
print("Decimal:", round(Decimal('0.70') * Decimal('1.05'), 2))
print("Float:", round(.70 * 1.05, 2))

The `Decimal` result keeps a trailing zero, automatically inferring four place significance from multiplicands with two place significance. Decimal reproduces mathematics as done by hand and avoids issues that can arise when binary floating point cannot exactly represent decimal quantities.

In [ ]:
# Modulo calculations
print("Decimal modulo:", Decimal('1.00') % Decimal('.10'))
print("Float modulo:", 1.00 % 0.10)

In [ ]:
# Equality tests
print("Decimal sum:", sum([Decimal('0.1')]*10) == Decimal('1.0'))
print("Float sum:", 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 + 0.1 == 1.0)

The `decimal` module provides arithmetic with as much precision as needed:

In [ ]:
getcontext().prec = 36
Decimal(1) / Decimal(7)